# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [22]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

d:\git\personal\udacity_aws_data_engineering\projects\1 - Data Modelling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [24]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Table Creation, Data Insertion and Query Validation

The newly generated `event_datafile_new.csv` will be used to insert data into tables. The following fields are available:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


### Apache Cassandra Configuration

#### Creating a Cluster
*Note:* a Docker Compose file has been provided to easily spin-up a Cassandra cluster. Simply run `docker compose up -d`. On Windows, it is recommended to use Docker Desktop with the WSL backend.

In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [26]:
keyspace = 'project1'
try:
    query = f"""
    CREATE KEYSPACE IF NOT EXISTS {keyspace}
    WITH replication = {{'class': 'SimpleStrategy', 'replication_factor' : 1}};
    """
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [27]:
try:
    session.set_keyspace(keyspace=keyspace)
except Exception as e:
    print(e)

#### Set Pandas Row Factory

In [28]:
# Define a custom row factory
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

# Set the row factory for Cassandra
session.row_factory = pandas_factory

In [29]:
# Placeholder for created tables
tables = []

## Create Tables to Run the Queries
Since Apache Cassandra is a NoSQL database, we must first consider the queries to model the tables after.

The following three questions are asked of the data:
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Queries must first be templated for each question, and tables created following.

### Query 1
> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Since we are working with NoSQL, it is imperative to first consider the query to model against.

In this case, the query would be structured follows:
`SELECT artist, song, length from table_name WHERE sessionId = 338 AND itemInSession = 4;`

Primary Key:
As mandated by the WHERE clause, the data model must include sessionId and itemInSession in the primary key definition.
Given that the combination of these two fields produce a unique value, they will be suitable candidates for the primary key.

Data Types:
By simple oberservation of the source data, we may infer some data types:

Numerical:
- sessionId (INT)
- itemInSession (INT)
- length (FLOAT)

Integers are given to fields with small, whole numbers, while the float is assigned to length as it contains fractional parts.

Non-Numerical:
- artist (VARCHAR)
- song (VARCHAR)

These text-based fields utilize VARCHAR as it is a more standard data type to use, but the TEXT data type may have also been used here.

In [30]:
# Model table after query requirements
table_name = 'music_history_by_session';
tables.append(table_name)

drop_query = f'DROP TABLE IF EXISTS {table_name};'

query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    length FLOAT,
    PRIMARY KEY (sessionId, itemInSession)
);
"""

try:
    session.execute(drop_query)
    session.execute(query)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(e)
    print(f"Error during table creation: {e}")

Table 'music_history_by_session' created successfully.


In [31]:
# Insert data into the table
file = 'event_datafile_new.csv'

tmp = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for row in csvreader:
        query = f"""
            INSERT INTO {table_name} (sessionId, itemInSession, artist, song, length) 
            VALUES (%s, %s, %s, %s, %s);
        """
        values = (int(row['sessionId']), int(row['itemInSession']), row['artist'], row['song'], float(row['length']))
        try:
            session.execute(query, values)
        except Exception as e:
            print(e)
            

In [32]:
# Verify the query results
table_name = 'music_history_by_session'
sessionId = 338
itemInSession = 4

query = f"""
SELECT artist, song, length
FROM {table_name}
WHERE sessionId = {sessionId}
AND itemInSession = {itemInSession};
"""

try:
    result_set = session.execute(query)
    df = result_set._current_rows
except Exception as e:
    print(e)

df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Again, we must first consider the query to model the table against.

In this case, the query would be structured follows:
`SELECT artist, song, user from table_name WHERE userId = 10 AND sessionId = 182;`

Primary Key:
- As mandated by the WHERE clause, the data model must include userId and sessionId in the primary key definition.
- In this case, we must also ensure that the song is sorted by itemInSession, and thus include it as a clustering column.
- For performance reasons, it would make sense to create a composite partition key of userId and sessionId. In this way, sessions for the same user will be stored on the same node. In a testing environment the implications are negligible, but it is important to consider how the data will behave in a production environment

Data Types:
Building on the data types outlined in query 1, we extend the following:

Numerical:
- userId (INT)

Non-Numerical:
- user (VARCHAR)

To note: the `user` field is a derived field from the `firstName` and `lastName` fields in the source data. Since these are both text-based, we assign a VARCHAR type to this new field.

In [33]:
# Model table after query requirements
table_name = 'music_history_by_user';
tables.append(table_name)

drop_query = f'DROP TABLE IF EXISTS {table_name};'

query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    user VARCHAR,
    PRIMARY KEY ((userId, sessionId), itemInSession)
);
"""

try:
    session.execute(drop_query)
    session.execute(query)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(e)
    print(f"Error during table creation: {e}")
                    

Table 'music_history_by_user' created successfully.


In [34]:
# Insert data into the table
file = 'event_datafile_new.csv'

tmp = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for row in csvreader:
        query = f"""
            INSERT INTO {table_name} (userId, sessionId, itemInSession, artist, song, user) 
            VALUES (%s, %s, %s, %s, %s, %s);
        """
        values = (
            int(row['userId']),
            int(row['sessionId']),
            int(row['itemInSession']),
            row['artist'],
            row['song'],
            f"{row['firstName']} {row['lastName']}"
        )
        try:
            session.execute(query, values)
        except Exception as e:
            print(e)
            

In [35]:
# Verify the query results
table_name = 'music_history_by_user'
userId = 10
sessionId = 182

query = f"""
SELECT artist, song, user
FROM {table_name}
WHERE userId = {userId}
AND sessionId = {sessionId};
"""

try:
    result_set = session.execute(query)
    df = result_set._current_rows
except Exception as e:
    print(e)

df.head()

,artist,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query 3
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this case, the query would be structured follows:
`SELECT user from table_name WHERE song = 'All Hands Against His Own;`

Primary Key:
- As mandated by the WHERE clause, the data model must at least include song in the primary key definition.
- Since we are also selecting the user, although not in the WHERE clause, we must enforce uniqueness on the primary key by including either the user or userId field.
- It would make the most sense to use the userId field in this case, in case of multiple users having the same name.
- Thus, the song field is used as the partition key, and the userId field as the clustering column.

In [36]:
# Model table after query requirements
table_name = 'user_history_by_song';
tables.append(table_name)

drop_query = f'DROP TABLE IF EXISTS {table_name};'

# Assume there is only one song, such that we do not need to specify the artist
query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    song VARCHAR,
    userId INT,
    user VARCHAR,
    PRIMARY KEY (song, userId)
);
"""

try:
    session.execute(drop_query)
    session.execute(query)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(e)
    print(f"Error during table creation: {e}")                   

Table 'user_history_by_song' created successfully.


In [37]:
# Insert data into the table
file = 'event_datafile_new.csv'

tmp = []
with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for row in csvreader:
        query = f"""
            INSERT INTO {table_name} (song, userId, user) 
            VALUES (%s, %s, %s);
        """
        values = (
            row['song'],
            int(row['userId']),
            f"{row['firstName']} {row['lastName']}"
        )
        try:
            session.execute(query, values)
        except Exception as e:
            print(e)

In [38]:
# Verify the query results
table_name = 'user_history_by_song'
song = 'All Hands Against His Own'

query = f"""
SELECT user
FROM {table_name}
WHERE song = '{song}';
"""

try:
    result_set = session.execute(query)
    df = result_set._current_rows
except Exception as e:
    print(e)

df.head()

,user
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Drop the tables before closing out the sessions

In [39]:
## TO-DO: Drop the table before closing out the sessions
for table in tables:
    query = f'DROP TABLE IF EXISTS {table};'
    try:
        session.execute(query)
        print(f"Table '{table}' dropped successfully.")
    except Exception as e:
        print(f"Error during table drop: {e}")

Table 'music_history_by_session' dropped successfully.
Table 'music_history_by_user' dropped successfully.
Table 'user_history_by_song' dropped successfully.


### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()